# Capstone Project
## Grocery Store's of the Greater DFW Area

This project will utilize multiple data sources including the US Census, Google Geocoder and Foursquare in order to build out a list of grocery stores in the Greater Dallas-Forth Worth Area. Once we have collected all the data and shaped it into a usuable dataset, we can begin to dig into the question on whether Denton, TX has cross a population threshold that would make a new Grocery Store a profitable investment.

In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import requests
import json
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import googlemaps

Import all the necessary packages to make web requests, read JSON, geocode locations, plot data, cluster the data and draw 
a map.

Will be querying the US Census to pull down the Population of every city in the following counties:

Dallas County (County Code 113),

Denton County (County Code 121),

Tarrant County (County Code 439)

In [2]:
df_denton = pd.read_csv("https://api.census.gov/data/2017/pep/population?get=POP,GEONAME&for=place:*&in=state:48%20county:121")
df_denton.rename(columns={'[["POP"':'Population', 'GEONAME':'City', 'place]':'Place'}, inplace=True)
df_denton['Population'] = df_denton['Population'].str[2:-1]
df_denton['City'] = df_denton['City'].str[:-22]
df_denton['Place'] = df_denton['Place'].str[:-1]
df_denton['City'] = df_denton['City'].str.strip('(pt.)')
df_denton['City'] = df_denton['City'].str.replace(' city', "")
df_denton['City'] = df_denton['City'].str.replace(' town', "")
df_denton.drop(columns=['state', 'Unnamed: 5', 'Place'], inplace=True)
df_denton = df_denton[:-1]
df_denton.head()

,Population,City,county
0,4100,Argyle,121
1,3391,Aubrey,121
2,1732,Bartonville,121
3,79715,Carrollton,121
4,0,Celina,121


The columns come in quite messy and need to be renamed.
We will rename the Population, City and Place columns, along with removing redundant information, and random artifacts in the "Pop" and "Place" Columns:

In [3]:
df_worth = pd.read_csv("https://api.census.gov/data/2017/pep/population?get=POP,GEONAME&for=place:*&in=state:48%20county:439")
df_worth.rename(columns={'[["POP"':'Population', 'GEONAME':'City', 'place]':'Place'}, inplace=True)
df_worth['Population'] = df_worth['Population'].str[2:-1]
df_worth['City'] = df_worth['City'].str[:-22]
df_worth['Place'] = df_worth['Place'].str[:-1]
df_worth['City'] = df_worth['City'].str.strip('(pt.),')
df_worth['City'] = df_worth['City'].str.replace(' city', "")
df_worth['City'] = df_worth['City'].str.replace(' town', "")
df_worth.drop(columns=['state', 'Unnamed: 5', 'Place'], inplace=True)
df_worth = df_worth[:-1]
df_worth.head()

,Population,City,county
0,396394,Arlington,439
1,10471,Azle,439
2,49486,Bedford,439
3,23590,Benbrook,439
4,2482,Blue Mound,439


We do the same process for Tarrant County

In [4]:
df_dal = pd.read_csv("https://api.census.gov/data/2017/pep/population?get=POP,GEONAME&for=place:*&in=state:48%20county:113")
df_dal.rename(columns={'[["POP"':'Population', 'GEONAME':'City', 'place]':'Place'}, inplace=True)
df_dal['Population'] = df_dal['Population'].str[2:-1]
df_dal['City'] = df_dal['City'].str[:-22]
df_dal['Place'] = df_dal['Place'].str[:-1]
df_dal['City'] = df_dal['City'].str.strip('(pt.)')
df_dal['City'] = df_dal['City'].str.replace(' city', "")
df_dal['City'] = df_dal['City'].str.replace(' town', "")
df_dal.drop(columns=['state', 'Unnamed: 5', 'Place'], inplace=True)
df_dal = df_dal[:-1]
df_dal.head()

,Population,City,county
0,15458,Addison,113
1,25357,Balch Springs,113
2,55993,Carrollton,113
3,48150,Cedar Hill,113
4,4251,Cockrell Hill,113


And Dallas county.

Now that the data is cleaned up a bit, its time to add the Longitude and Latitude for each city.

In [40]:
# The code was removed by Watson Studio for sharing.

In [6]:
def getCityGeo( city, area ):
    latitude = []
    longitude = []
    cities = city['City'].values

    for c in cities:
        lat_lng_coords = None
        geojson = None
        while(lat_lng_coords is None):
            gmaps = googlemaps.Client(key=API_key)
            #g = geocoder.google(area.format(c))
            lat_lng_coords = gmaps.geocode(area.format(c))
            
    
    
        latitude.append(lat_lng_coords[0]['geometry']['location']['lat'])
        longitude.append(lat_lng_coords[0]['geometry']['location']['lng'])
        #latitude.append(lat_lng_coords[0])
        #longitude.append(lat_lng_coords[1])
    
    city['Latitude'] = latitude
    city['Longitude'] = longitude
    return city

Run the function for all 3 counties to attach the longitude and latitude to the data set.

In [7]:
getCityGeo(df_denton,'{}, Denton County, Texas')
getCityGeo(df_worth,'{}, Tarrant County, Texas')
getCityGeo(df_dal,'{}, Dallas County, Texas')

,Population,City,county,Latitude,Longitude
0,15458,Addison,113,32.961790,-96.829169
1,25357,Balch Springs,113,32.728741,-96.622771
2,55993,Carrollton,113,32.975642,-96.889964
3,48150,Cedar Hill,113,32.588469,-96.956115
4,4251,Cockrell Hill,113,32.736242,-96.886948
5,788,Combine,113,32.588468,-96.508599
6,41119,Coppell,113,32.954569,-97.015008
7,1258325,Dallas,113,32.776664,-96.796988
8,53568,DeSoto,113,32.589700,-96.857074
9,39487,Duncanville,113,32.651800,-96.908337


In [8]:
df_dal.head()

,Population,City,county,Latitude,Longitude
0,15458,Addison,113,32.961790,-96.829169
1,25357,Balch Springs,113,32.728741,-96.622771
2,55993,Carrollton,113,32.975642,-96.889964
3,48150,Cedar Hill,113,32.588469,-96.956115
4,4251,Cockrell Hill,113,32.736242,-96.886948


In [9]:
df_denton.head()

,Population,City,county,Latitude,Longitude
0,4100,Argyle,121,33.121232,-97.183347
1,3391,Aubrey,121,33.304283,-96.986118
2,1732,Bartonville,121,33.073177,-97.131679
3,79715,Carrollton,121,32.975642,-96.889964
4,0,Celina,121,33.366454,-96.764097


In [10]:
df_worth.head()

,Population,City,county,Latitude,Longitude
0,396394,Arlington,439,32.735687,-97.108066
1,10471,Azle,439,32.895126,-97.545856
2,49486,Bedford,439,32.844017,-97.143067
3,23590,Benbrook,439,32.673188,-97.460576
4,2482,Blue Mound,439,32.856517,-97.338906


The final step for preparing the initial city dataset is to join together all 3 dataframes into a single data frame.

In [11]:
df_w = pd.concat([df_denton, df_worth, df_dal], ignore_index=True)
df_w.head()

,Population,City,county,Latitude,Longitude
0,4100,Argyle,121,33.121232,-97.183347
1,3391,Aubrey,121,33.304283,-96.986118
2,1732,Bartonville,121,33.073177,-97.131679
3,79715,Carrollton,121,32.975642,-96.889964
4,0,Celina,121,33.366454,-96.764097


# Foursquare Setup

In [39]:
# The code was removed by Watson Studio for sharing.

In [13]:
def getNearbyGroceryVenues(names, latitudes, longitudes, pop, categoryId = '4bf58dd8d48988d1f9941735,4bf58dd8d48988d118951735' ):
    grocery_list=[]
    for name, lat, lng, pop in zip(names, latitudes, longitudes, pop):
            
   
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&limit={}&radius={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng, 
            API_VERSION,  
            LIMIT,
            radius,
            categoryId)


        results = requests.get(url).json()['response']['venues']
        try:
            x=(results[0]['categories'][0]['id'])
        except:
            x='Nan'
        try:            
            y=(results[0]['categories'][0]['name'])
        except:
            y='Nan'
        try:
            z=(results[0]['location']['postalCode'])
        except:
            z='Nan'
        grocery_list.append([(x, name, lat, lng, v['name'], v['location']['lat'], v['location']['lng'], y, v['location']['distance'], z, pop) for v in results])
            
    dfw_groceryvenues = pd.DataFrame([item for grocery_list in grocery_list for item in grocery_list])
    dfw_groceryvenues.columns = ['Id','City', 'City Latitude', 'City Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category', 'Distance', 'Zip Code', 'Population']
    #dfw_groceryvenues['Population'] = name['Population']
    
    return(dfw_groceryvenues)

Call the "getNearbyGroceryVenues" function.

In [14]:
dfw_groceryvenues = getNearbyGroceryVenues(names=df_w['City'], latitudes=df_w['Latitude'], longitudes=df_w['Longitude'], pop=df_w['Population'])

In [15]:
dfw_groceryvenues.head()

,Id,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance,Zip Code,Population
0,4bf58dd8d48988d1f9941735,Argyle,33.121232,-97.183347,Hidden Valley Dairy,33.117789,-97.196208,Food & Drink Shop,1258,76226,4100
1,4bf58dd8d48988d1f9941735,Argyle,33.121232,-97.183347,Boydston Foods,33.151838,-97.164065,Food & Drink Shop,3852,76226,4100
2,4bf58dd8d48988d118951735,Aubrey,33.304283,-96.986118,Diamond Supermarket,33.293934,-96.984347,Grocery Store,1163,76227,3391
3,4bf58dd8d48988d118951735,Aubrey,33.304283,-96.986118,Diamond Foods,33.293752,-96.984574,Grocery Store,1181,76227,3391
4,52f2ab2ebcbc57f1066b8b46,Bartonville,33.073177,-97.131679,Kroger Lantana Town Center,33.074904,-97.130014,Supermarket,247,Nan,1732


In [16]:
dfw_groceryvenues.groupby('City').count().sort_values(by='Zip Code', ascending=False).head()

,Id,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance,Zip Code,Population
City,,,,,,,,,,
Dallas,90,90,90,90,90,90,90,90,90,90
Fort Worth,74,74,74,74,74,74,74,74,74,74
Grapevine,68,68,68,68,68,68,68,68,68,68
Carrollton,60,60,60,60,60,60,60,60,60,60
Addison,50,50,50,50,50,50,50,50,50,50


In [17]:
dfw_groceryvenues.drop('Id', axis=1, inplace=True)

We create a new dataframe at this point, 'dfw_data' which we will be using the k-means algorithm on.

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
dfw_onehot = pd.get_dummies(dfw_groceryvenues[['Venue']], prefix = '', prefix_sep='')
dfw_onehot['City']=dfw_groceryvenues['City']
fixed_columns = [dfw_onehot.columns[-1]]+list(dfw_onehot.columns[:-1])
dfw_onehot= dfw_onehot[fixed_columns]
dfw_onehot.shape
#grouping by neighbourhood
dfw_grouped=dfw_onehot.groupby('City').mean().reset_index()
dfw_grouped

,City,1 Stop Beer Wine & Cigar,1001 Wine & Spirits,5th Street Market,6 To 12 Food Store,6S Liquor,7-Eleven,7mart,99 Ranch Market,A Plus,...,india imports,krispy kreme,la azteca meat market,lewisville liquor,nutrition clubhouse,quickway,shaved ice snow cones,shax',shop and go,whistle stop
0,Addison,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,Argyle,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,Arlington,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,Aubrey,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,Azle,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
5,Balch Springs,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
6,Bartonville,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
7,Bedford,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
8,Benbrook,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
9,Blue Mound,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [22]:
#sorting in descending order
num_top_venues=5
indicators=['st','nd','rd']
columns=['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append("{}{} Most Common Venue".format(ind+1,indicators[ind]))
    except:
        columns.append("{}th Most Common Venue".format(ind+1))
#new dataframe
dfw_venuessorted = pd.DataFrame(columns=columns)
dfw_venuessorted['City']=dfw_grouped['City']
for ind in np.arange(dfw_grouped.shape[0]):
    dfw_venuessorted.iloc[ind,1:] = return_most_common_venues(dfw_grouped.iloc[ind, :], num_top_venues)
dfw_venuessorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Addison,Tom Thumb,GNC,Culinary Art Catering,Stuart's Exquisite Services LLC,Barrel King
1,Argyle,Boydston Foods,Hidden Valley Dairy,whistle stop,Friendly's,General Parts Group
2,Arlington,Family Dollar,QuikTrip,Kroger,GNC,Dana Bazaar Indian Groceries
3,Aubrey,Diamond Foods,Diamond Supermarket,Freshomatic USA,Garden Ridge Farmers Market,GNC - Valley View Mall
4,Azle,Apex Liquor,Walmart Supercenter,Majestic Liquor,"Majestic Spirits, Beer & Wine",Farmer's Market


We create a new dataframe at this point, 'dfw_data' which we will be using the k-means algorithm on.

In [23]:
dfw_data = df_w
store_density = pd.DataFrame(dfw_groceryvenues['City'].value_counts().reset_index().rename(columns={'index': 'city', 'City': 'count'}))
store_density = store_density.sort_values(by='city', ascending=True)
store_density['Zip Code'] = dfw_groceryvenues['Zip Code']
dfw_data = dfw_data.join(store_density.set_index('city'), on='City')
dfw_data = dfw_data[['Population', 'City', 'Latitude', 'Longitude', 'count', 'county', 'Zip Code']]
dfw_data = dfw_data.rename(columns={'count' : 'Store Count'})
dfw_data = dfw_data.dropna()
dfw_data['Population'] = pd.to_numeric(dfw_data['Population'], errors='coerce')
dfw_data['Zip Code'] = pd.to_numeric(dfw_data['Zip Code'], errors='coerce')
dfw_data['Popdense'] = dfw_data['Population']/dfw_data['Store Count']
dfw_data = dfw_data.dropna()
dfw_data.head()

,Population,City,Latitude,Longitude,Store Count,county,Zip Code,Popdense
0,4100,Argyle,33.121232,-97.183347,2.0,121,76205.0,2050.000000
1,3391,Aubrey,33.304283,-96.986118,2.0,121,76205.0,1695.500000
2,1732,Bartonville,33.073177,-97.131679,5.0,121,76205.0,346.400000
3,79715,Carrollton,32.975642,-96.889964,60.0,121,76227.0,1328.583333
5,822,Coppell,32.954569,-97.015008,20.0,121,75019.0,41.100000


# K-Means Clustering

In [24]:
# set number of clusters
kclusters = 10

dfw_grouped_clustering = dfw_data.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfw_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([6, 6, 6, 4, 6, 6, 6, 6, 9, 8], dtype=int32)

In [25]:
dfw_merged = dfw_data

# add clustering labels
dfw_merged['Cluster Labels'] = kmeans.labels_

dfw_merged = dfw_merged.join(dfw_venuessorted.set_index('City'), on='City')
dfw_merged.head()

,Population,City,Latitude,Longitude,Store Count,county,Zip Code,Popdense,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,4100,Argyle,33.121232,-97.183347,2.0,121,76205.0,2050.000000,6,Boydston Foods,Hidden Valley Dairy,whistle stop,Friendly's,General Parts Group
1,3391,Aubrey,33.304283,-96.986118,2.0,121,76205.0,1695.500000,6,Diamond Foods,Diamond Supermarket,Freshomatic USA,Garden Ridge Farmers Market,GNC - Valley View Mall
2,1732,Bartonville,33.073177,-97.131679,5.0,121,76205.0,346.400000,6,Kroger Marketplace,Bartonville Food Store,Lone Star Country Corner Stop,Stir-ups Liquors,Kroger Lantana Town Center
3,79715,Carrollton,32.975642,-96.889964,60.0,121,76227.0,1328.583333,4,Walmart Supercenter,Walmart Garden Center,Kelly's Beer & Wine,Tatering,GNC
5,822,Coppell,32.954569,-97.015008,20.0,121,75019.0,41.100000,6,Tom Thumb,Coppell Farmers Market,GNC,Bountiful baskets,QuikTrip


In [26]:
add_dfw = 'DFW'

geolocator_dfw = Nominatim()
location_dfw = geolocator_dfw.geocode(add_dfw)
latitude_dfw = location_dfw.latitude
longitude_dfw = location_dfw.longitude
print('The geograpical coordinate of DFW Area are {}, {}.'.format(latitude_dfw, longitude_dfw))

The geograpical coordinate of DFW Area are 32.89651945, -97.0465220537124.


## Mapping the K-Means Output

In [27]:
# visulaize clusters
map_clusters=folium.Map(location=[latitude_dfw,longitude_dfw],zoom_start=12)
x = np.arange(kclusters)
ys=[i+x+(i*x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
#add markers
markers_colors=[]
for lat, lon, poi, cluster in zip(dfw_merged['Latitude'],dfw_merged['Longitude'], dfw_merged['City'], dfw_merged['Cluster Labels']):
    label=folium.Popup(str(poi)+' Cluster '+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

# Reviewing the individual Cluster Types

Cluster 0 seems to be focused on the places with the least population or fewest stores.

In [28]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 0, dfw_merged.columns[[1] + list(range(5, dfw_merged.shape[1]))]]

,City,county,Zip Code,Popdense,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Highland Village,121,75077.0,975.705882,0,Goody Goody Liquor,Walmart Supercenter,"Majestic Spirits, Beer & Wine",Edible Arrangements,Tom Thumb
43,Trophy Club,121,75019.0,538.545455,0,Walmart Grocery Pickup,Sun Fresh Market,Roanoke Nutrition,Mumbai Place,"Majestic Spirits, Beer & Wine"
48,Benbrook,439,76205.0,3931.666667,0,Walmart Grocery Pickup,Family Dollar,Hugo's Liquor,QuikTrip,Walmart Supercenter
51,Colleyville,439,75006.0,1212.454545,0,Tom Thumb,Walmart Grocery Pickup,Tivoli Wines and Spirits,Mirage Fine Spirits,Walmart Neighborhood Market
52,Crowley,439,75077.0,1279.500000,0,Walmart Grocery Pickup,front end of krogers,Walmart Supercenter,Family Dollar,Empire Nutrition
58,Forest Hill,439,75077.0,761.941176,0,Family Dollar,QuikTrip,Bibere Beverages,Foodland,Prime Valley Liquor
78,Saginaw,439,75006.0,821.928571,0,Kroger,QuikTrip,Kroger Marketplace- Saginaw,Big Daddys Liquor,Big Daddy's Fine Wine & Spirits
82,Watauga,439,75006.0,745.515152,0,QuikTrip,Family Dollar,Albertsons,GNC,Watauga Farmer's Market
86,White Settlement,439,75006.0,614.758621,0,QuikTrip,Albertsons,Walmart Supercenter,Family Dollar,Walmart Grocery Pickup
88,Balch Springs,113,75006.0,1056.541667,0,Family Dollar,Walmart Neighborhood Market,Liquor Depot,Mom's Grocery,QuikTrip


Cluster 1 has a much higher population density.

In [29]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 1, dfw_merged.columns[[1] + list(range(5, dfw_merged.shape[1]))]]

,City,county,Zip Code,Popdense,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
59,Fort Worth,439,76226.0,11687.513514,1,Family Dollar,Simply Fit Meals,QuikTrip,Burgundy's Local,Natural Grocers


In [30]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 2, dfw_merged.columns[[1] + list(range(5, dfw_merged.shape[1]))]]

,City,county,Zip Code,Popdense,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
99,Garland,113,75006.0,7201.424242,2,Family Dollar,Walmart Neighborhood Market,QuikTrip,Walmart Pharmacy,Ohana Hawaiian BBQ
105,Irving,113,75006.0,8288.724138,2,Family Dollar,Fiesta Mart,QuikTrip,Kroger,ALDI


Cluster 3 is for Dallas, which stands out as the most population and the most stores.

In [31]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 3, dfw_merged.columns[[1] + list(range(5, dfw_merged.shape[1]))]]

,City,county,Zip Code,Popdense,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
94,Dallas,113,76226.0,13981.388889,3,Family Dollar,ALDI,Kroger,In the Sack,Market Provisions Co.


In [32]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 4]#, dfw_merged.columns[[1] + list(range(5, dfw_merged.shape[1]))]]

,Population,City,Latitude,Longitude,Store Count,county,Zip Code,Popdense,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,79715,Carrollton,32.975642,-96.889964,60.0,121,76227.0,1328.583333,4,Walmart Supercenter,Walmart Garden Center,Kelly's Beer & Wine,Tatering,GNC
14,76414,Flower Mound,33.014567,-97.096955,24.0,121,75006.0,3183.916667,4,Tom Thumb,Michael Owens Catering,Sprouts Farmers Market,Walmart Neighborhood Market,Tarkari
16,68052,Frisco,33.150674,-96.823612,25.0,121,75006.0,2722.080000,4,Kroger,QuikTrip,GNC,Market Street,Vitality Bowls Frisco
69,66755,Mansfield,32.563192,-97.141677,12.0,439,75077.0,5562.916667,4,QuikTrip,Tom Thumb,Dr Kimberli Omwanghe M.D.,Walmart Grocery Pickup,WineStyles
71,70441,North Richland Hills,32.834295,-97.228903,34.0,439,75006.0,2071.794118,4,QuikTrip,Family Dollar,GNC,Walmart Neighborhood Market,Kroger


In [33]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 5]#, dfw_merged.columns[[0] + list(range(5, dfw_merged.shape[1]))]]

,Population,City,Latitude,Longitude,Store Count,county,Zip Code,Popdense,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
45,396394,Arlington,32.735687,-97.108066,44.0,439,75006.0,9008.954545,5,Family Dollar,QuikTrip,Kroger,GNC,Dana Bazaar Indian Groceries


In [34]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 6]#, dfw_merged.columns[[1] + list(range(5, dfw_merged.shape[1]))]]

,Population,City,Latitude,Longitude,Store Count,county,Zip Code,Popdense,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,4100,Argyle,33.121232,-97.183347,2.0,121,76205.0,2050.000000,6,Boydston Foods,Hidden Valley Dairy,whistle stop,Friendly's,General Parts Group
1,3391,Aubrey,33.304283,-96.986118,2.0,121,76205.0,1695.500000,6,Diamond Foods,Diamond Supermarket,Freshomatic USA,Garden Ridge Farmers Market,GNC - Valley View Mall
2,1732,Bartonville,33.073177,-97.131679,5.0,121,76205.0,346.400000,6,Kroger Marketplace,Bartonville Food Store,Lone Star Country Corner Stop,Stir-ups Liquors,Kroger Lantana Town Center
5,822,Coppell,32.954569,-97.015008,20.0,121,75019.0,41.100000,6,Tom Thumb,Coppell Farmers Market,GNC,Bountiful baskets,QuikTrip
6,1457,Copper Canyon,33.095955,-97.096678,9.0,121,76226.0,161.888889,6,Walmart Grocery Pickup,Vitamin Shoppe,GNC,Whole Foods Market,World Market
8,45,Corral City,33.100122,-97.226127,3.0,121,76205.0,15.000000,6,Hidden Valley Dairy,Paradise Liquors,Paradise Spirits & Fine Wines,whistle stop,Friendly's
9,1302,Cross Roads,33.231828,-97.002291,5.0,121,76205.0,260.400000,6,Walmart Supercenter,Walmart Grocery Pickup,Jacob's Liquors,Walmart Garden Center,GNC
13,3079,Double Oak,33.065122,-97.110567,12.0,121,75077.0,256.583333,6,GNC,Sub Zero Nitrogen Ice Cream,World Market,Kroger Lantana Town Center,Whole Foods Market
15,9284,Fort Worth,32.755488,-97.330766,74.0,121,76226.0,125.459459,6,Family Dollar,Simply Fit Meals,QuikTrip,Burgundy's Local,Natural Grocers
17,0,Grapevine,33.003765,-96.870178,68.0,121,76227.0,0.000000,6,QuikTrip,GNC,Walmart Supercenter,Tom Thumb,The Spice & Tea Exchange


In [35]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 7]#, dfw_merged.columns[[1] + list(range(5, dfw_merged.shape[1]))]]

,Population,City,Latitude,Longitude,Store Count,county,Zip Code,Popdense,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
47,49486,Bedford,32.844017,-97.143067,20.0,439,75019.0,2474.300000,7,Kroger,Family Dollar,QuikTrip,GNC,Foodland
55,55174,Euless,32.837073,-97.081954,27.0,439,75006.0,2043.481481,7,QuikTrip,Family Dollar,Scratch Cafe & Shop,Bombay Grocery & +,Albertsons
60,60284,Grand Prairie,32.745964,-96.997785,34.0,439,75006.0,1773.058824,7,Family Dollar,QuikTrip,El Rio Grande Latin Market,Terry's Mercado,Kroger
61,53982,Grapevine,32.934292,-97.078065,68.0,439,76227.0,793.852941,7,QuikTrip,GNC,Walmart Supercenter,Tom Thumb,The Spice & Tea Exchange
65,47266,Keller,32.934189,-97.229298,10.0,439,76210.0,4726.600000,7,GNC,Super Suppers,Natural Grocers,QuikTrip,Tom Thumb
89,55993,Carrollton,32.975642,-96.889964,60.0,113,76227.0,933.216667,7,Walmart Supercenter,Walmart Garden Center,Kelly's Beer & Wine,Tatering,GNC
90,48150,Cedar Hill,32.588469,-96.956115,13.0,113,75077.0,3703.846154,7,Family Dollar,Walmart Grocery Pickup,ALDI,GNC,Tastee Plates Catering
95,53568,DeSoto,32.589700,-96.857074,18.0,113,75019.0,2976.000000,7,ALDI,Walmart Supercenter,Family Dollar,Walmart Neighborhood Market,The Daiquiri Factory and Cafe
111,55286,Rowlett,32.902902,-96.563880,13.0,113,75077.0,4252.769231,7,QuikTrip,Walmart Garden Center,Goody Goody,Sprouts Farmers Market,Walmart Grocery Pickup


## Denton

The goal was to see whether Denton needs a new store at this time. It seems these numbers suggest that it is a prime location to build a new store based on the optimal ratio of Population and Store availability.


In [36]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 8]#, dfw_merged.columns[[1] + list(range(5, dfw_merged.shape[1]))]]

,Population,City,Latitude,Longitude,Store Count,county,Zip Code,Popdense,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,136268,Denton,33.214841,-97.133068,45.0,121,75006.0,3028.177778,8,Kroger,La Azteca,Austin Street Truck Stop,Market On Oak,Ken's Produce
101,133525,Grand Prairie,32.745964,-96.997785,34.0,113,75006.0,3927.205882,8,Family Dollar,QuikTrip,El Rio Grande Latin Market,Terry's Mercado,Kroger
108,143818,Mesquite,32.766796,-96.599159,21.0,113,75019.0,6848.476190,8,Family Dollar,QuikTrip,Albertsons,LAREDO MEAT MARKET,Taj Mahal


In [37]:
dfw_merged.loc[dfw_merged['Cluster Labels'] == 9]#, dfw_merged.columns[[1] + list(range(5, dfw_merged.shape[1]))]]

,Population,City,Latitude,Longitude,Store Count,county,Zip Code,Popdense,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,30503,Dallas,33.199165,-97.123983,90.0,121,76226.0,338.922222,9,Family Dollar,ALDI,Kroger,In the Sack,Market Provisions Co.
42,42721,The Colony,33.080608,-96.892831,20.0,121,75019.0,2136.050000,9,The Thirsty Growler,Kroger Marketplace,Spec's,Water Inn,Walmart Supercenter
62,44417,Haltom City,32.799574,-97.269182,28.0,439,75006.0,1586.321429,9,Family Dollar,Walmart Neighborhood Market,QuikTrip,Fort Worth Discount Liquor,Trav's Liqour Store
64,39051,Hurst,32.823462,-97.170568,22.0,439,75006.0,1775.045455,9,Walmart Grocery Pickup,QuikTrip,GNC,Walmart Neighborhood Market,Kroger
80,30902,Southlake,32.941236,-97.134178,40.0,439,75006.0,772.550000,9,Tom Thumb,GNC,Fresh Market,Fresh Fit Foods,WineStyles
93,41119,Coppell,32.954569,-97.015008,20.0,113,75019.0,2055.950000,9,Tom Thumb,Coppell Farmers Market,GNC,Bountiful baskets,QuikTrip
96,39487,Duncanville,32.651800,-96.908337,31.0,113,75006.0,1273.774194,9,Family Dollar,QuikTrip,Walmart Supercenter,ALDI,Red Bird Dialysis
97,37088,Farmers Branch,32.926514,-96.896115,22.0,113,75019.0,1685.818182,9,QuikTrip,Walmart Neighborhood Market,La Azteca Meat Market,Ko'mart Marketplace,"KETTLE-COOKED CRAFTERS, LLC"
